In [1]:
import Data.Bits
import Data.Vector (Vector)
import qualified Data.Vector as V
import Data.Word
import Prelude hiding (lookup)

type Hash = Word64
type Bitmap = Word64
type Shift = Int

class Hashable a where
    hash :: a -> Hash

data HAMT key value
    = None
    | Leaf Hash key value
    | Many Bitmap (Vector (HAMT key value))
    deriving (Show)

In [2]:
bitsPerSubkey :: Int
bitsPerSubkey = 4

subkeyMask :: Bitmap
subkeyMask = 1 `shiftL` bitsPerSubkey - 1

subkey :: Hash -> Shift -> Int
subkey hash shift = fromIntegral $ shiftR hash shift .&. subkeyMask

maskIndex :: Bitmap -> Bitmap -> Int
maskIndex bitmap mask = popCount (bitmap .&. (mask - 1))

bitMask :: Hash -> Shift -> Bitmap
bitMask hash shift = 1 `shiftL` (subkey hash shift)

In [3]:
lookup :: (Eq key, Hashable key) => key -> HAMT key value -> Maybe value
lookup key hamt = lookup' 0 (hash key) key hamt

lookup' :: (Eq key) => Shift -> Hash -> key -> HAMT key value -> Maybe value
lookup' shift hash key hamt = case hamt of
    None -> Nothing
    Leaf leafHash leafKey leafValue -> if hash == leafHash && key == leafKey
        then Just leafValue
        else Nothing
    Many bitmap vector -> let
        mask = bitMask hash shift
        in if bitmap .&. mask == 0
            then Nothing
            else lookup' (shift+bitsPerSubkey) hash key (vector V.! (maskIndex bitmap mask))

In [4]:
insert :: (Eq key, Hashable key) => key -> value -> HAMT key value -> HAMT key value
insert key value hamt = insert' 0 (hash key) key value hamt

insert' :: (Eq key) => Shift -> Hash -> key -> value -> HAMT key value -> HAMT key value
insert' shift hash key value hamt = case hamt of
    None -> Leaf hash key value
    Leaf leafHash leafKey leafValue -> if hash == leafHash && key == leafKey
        then Leaf hash key value
        else insert' shift hash key value (Many (bitMask leafHash shift) (V.singleton hamt))
    Many bitmap vector -> let
        mask = bitMask hash shift
        index = maskIndex bitmap mask
        in if bitmap .&. mask == 0
            then let
                leaf = Leaf hash key value
                vector' = V.take index vector V.++ V.singleton leaf V.++ V.drop index vector
                bitmap' = bitmap .|. mask
                in Many bitmap' vector'
            else let
                subtree = vector V.! index
                subtree' = insert' (shift+bitsPerSubkey) hash key value subtree
                vector' = vector V.// [(index, subtree')]
                in Many bitmap vector'

In [5]:
delete :: (Eq key, Hashable key) => key -> HAMT key value -> HAMT key value
delete key hamt = delete' 0 (hash key) key hamt

delete' :: (Eq key) => Shift -> Hash -> key -> HAMT key value -> HAMT key value
delete' shift hash key hamt = case hamt of
    None -> None
    Leaf leafHash leafKey leafValue -> if hash == leafHash && key == leafKey
        then None
        else hamt
    Many bitmap vector -> let
        mask = bitMask hash shift
        index = maskIndex bitmap mask
        in if bitmap .&. mask == 0
            then hamt
            else let
                subtree = vector V.! index
                subtree' = delete' (shift+bitsPerSubkey) hash key subtree
                in case subtree' of
                    None -> if V.length vector == 1
                        then None
                        else Many (bitmap .&. complement mask) (V.take index vector V.++ V.drop (index+1) vector)
                    l@Leaf{} -> if V.length vector == 1
                        then l
                        else Many bitmap (vector V.// [(index, subtree')])
                    _ ->     Many bitmap (vector V.// [(index, subtree')])

In [6]:
{-# LANGUAGE OverloadedStrings #-}

import Numeric
import Data.Char

import qualified Data.ByteString as B

import Data.ByteArray.Hash

instance Hashable B.ByteString where
    hash bs = let
        FnvHash64 h = fnv1_64Hash bs
        in h

empty :: HAMT k v
empty = None

t = insert ("2" :: B.ByteString) 2 $ insert ("1" :: B.ByteString) 1 empty
t' = insert ("3" :: B.ByteString) 3 $ delete ("2" :: B.ByteString) t
lookup ("1" :: B.ByteString) t

Just 1